## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
# application_df.head()
application_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(["EIN","NAME"],1)

C:\Users\madis\AppData\Local\Temp/ipykernel_15284/1538362199.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(["EIN","NAME"],1)


In [3]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
apptype_count=application_df.APPLICATION_TYPE.value_counts()
apptype_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
application_types_to_replace = list(apptype_count[apptype_count < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
classification_count=application_df.CLASSIFICATION.value_counts()
print(classification_count)
print("-"*50)
print(f"Average: {classification_count.mean()}")
print(f"Median: {classification_count.median()}") 

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64
--------------------------------------------------
Average: 483.0845070422535
Median: 5.0


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
clfs_to_remove = pd.DataFrame(classification_count.loc[lambda x : x == 1])
clfs_to_remove_list = clfs_to_remove.index.tolist()


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace = list(classification_count[classification_count < 500 ].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [35]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
application_df_dummies = pd.get_dummies(application_df,
                                        columns = ["APPLICATION_TYPE", 
                                                   "AFFILIATION", 
                                                   "CLASSIFICATION", 
                                                   "USE_CASE", 
                                                   "ORGANIZATION", 
                                                   "INCOME_AMT", 
                                                   "SPECIAL_CONSIDERATIONS"], 
                                        drop_first = True)


In [10]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = application_df_dummies.IS_SUCCESSFUL
X = application_df_dummies.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
input_feature_count = len(X_train_scaled[0])
input_feature_count

37

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
input_feature_count = len(X_train_scaled[0])
input_feature_count

hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="sigmoid", input_dim = input_feature_count))

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Added a third hidden layer to improve accuracy
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="softmax"))

# Added a fourth hidden layer to improve accuracy
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 150)               5700      
                                                                 
 dense_6 (Dense)             (None, 50)                7550      
                                                                 
 dense_7 (Dense)             (None, 10)                510       
                                                                 
 dense_8 (Dense)             (None, 10)                110       
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 13,881
Trainable params: 13,881
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
#  YOUR CODE GOES HERE
#  nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 737us/step - loss: 0.5909 - accuracy: 0.7084
Epoch 2/100
804/804 [==============================] - 1s 748us/step - loss: 0.5697 - accuracy: 0.7260
Epoch 3/100
804/804 [==============================] - 1s 735us/step - loss: 0.5619 - accuracy: 0.7285
Epoch 4/100
804/804 [==============================] - 1s 736us/step - loss: 0.5606 - accuracy: 0.7293
Epoch 5/100
804/804 [==============================] - 1s 736us/step - loss: 0.5591 - accuracy: 0.7285
Epoch 6/100
804/804 [==============================] - 1s 734us/step - loss: 0.5571 - accuracy: 0.7294
Epoch 7/100
804/804 [==============================] - 1s 736us/step - loss: 0.5564 - accuracy: 0.7299
Epoch 8/100
804/804 [==============================] - 1s 740us/step - loss: 0.5547 - accuracy: 0.7292
Epoch 9/100
804/804 [==============================] - 1s 735us/step - loss: 0.5544 - accuracy: 0.7306
Epoch 10/100
804/804 [==============================] - 1s 750us/step - l

804/804 [==============================] - 1s 736us/step - loss: 0.5371 - accuracy: 0.7394
Epoch 80/100
804/804 [==============================] - 1s 732us/step - loss: 0.5364 - accuracy: 0.7399
Epoch 81/100
804/804 [==============================] - 1s 735us/step - loss: 0.5368 - accuracy: 0.7389
Epoch 82/100
804/804 [==============================] - 1s 732us/step - loss: 0.5363 - accuracy: 0.7399
Epoch 83/100
804/804 [==============================] - 1s 735us/step - loss: 0.5363 - accuracy: 0.7393
Epoch 84/100
804/804 [==============================] - 1s 731us/step - loss: 0.5361 - accuracy: 0.7402
Epoch 85/100
804/804 [==============================] - 1s 736us/step - loss: 0.5365 - accuracy: 0.7400
Epoch 86/100
804/804 [==============================] - 1s 733us/step - loss: 0.5360 - accuracy: 0.7400
Epoch 87/100
804/804 [==============================] - 1s 737us/step - loss: 0.5362 - accuracy: 0.7405
Epoch 88/100
804/804 [==============================] - 1s 732us/step - loss:

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5548 - accuracy: 0.7293 - 201ms/epoch - 749us/step
Loss: 0.5548244714736938, Accuracy: 0.7293294668197632


In [20]:
len(nn.get_weights())

10

In [21]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn.save("AlphabetSoupCharity_Optimization.h5")